# Ch05. XGBoost
**구성**
1. XGBoost가 트리 앙상블 알고리즘을 향상시킨 이론(XGBoost의 속도 향상, 누락된 값 처리 방법, 규제 매개변수의 수학이론)
    + XGBoost 구조
    + XGBoost 매개변수 분석
2. 힉스 보손 캐글 대회
    + XGBoost 모델 만들기(분류, 회귀)
    + 대회 사례 연구

## 5.1. XGBoost 구조
XGBoost는 Gradient 부스팅을 크게 업그레이드한 모델

### 5.1.1. 역사
+ **결정트리**의 경우 가지치기 없이 모든 리프 노드가 순수노드가 되는 경우, 새로운 데이터에 대한 일반화 성능이 떨어집니다.
+ **앙상블 방법**은 **배깅**과 **부스팅**을 통해 많은 결정 트리를 연결하기 때문에 더 효과적
    + 앙상블 중에 선두 알고리즘은 Gradient Boosting
+ 워싱턴 대학의 티엔치 첸은 Gradient Boosting의 일관성, 성능, 뛰어난 결과를 더욱 향상 시킨, **익스트림 그레디언트 부스팅 : XGBoost**를 고안함
    + 내장된 규제와 속도 향상이 포함됨

### 5.1.2. 주요 기능
**XGBoost**는 계산 능력을 극대화한다는 의미로, 이를 위해서는 모델 구축뿐 아니라 디스크 입출력, 압축, 캐싱, cpu에 대한 지식이 필요

#### 1) 누락된 값 처리
XGBoost는 자체적으로 누락된 값을 처리 가능
+ `missing` 매개변수에 값을 지정 가능
    + XGBoost는 누락 값을 좌/우측으로 보내는 분할들 중 최선의 결과를 내는 분할을 선택
    
#### 2) 속도향상
XGBoost는 특히 속도에 주안점을 두고 설계됨. 다음과 같은 기능이 속도를 향상시킴
+ 근사 분할 탐색 알고리즘
+ 희소성 고려 분할 탐색
+ 병렬 컴퓨팅
+ 캐시 고려 접근
+ 블록 압축과 샤딩

#### 2-1) 근사 분할 탐색 알고리즘
이는 데이터를 나누는 퍼센트인 분위수를 사용하여 후보 분할을 제안함
+ 전역 제안(global proposal) : 동일한 분위수가 전체 훈련에 사용됨
+ 지역 제안(local proposal) : 각 분할마다 새로운 분위수를 제안

콴타일 스케치 알고리즘(quantile sketch algorithm)은 가중치가 균일한 데이터 셋에서 잘 작동함. 
+ XGBoost는 가중 콴타일 스케치를 사용함

#### 2-2) 희소성 고려 분할 탐색
분할을 탐색할 때, 희소한 행렬에서 XGBoost가 더 빠르게 동작하게 됨

#### 2-3) 병렬컴퓨팅
부스팅은 각 트리가 이전 트리의 결과에 의존하기 때문에 병렬 컴퓨팅에 이상적이지 않지만, 병렬화가 가능한 부분이 있음
+ XGBoost는 데이터를 블록(block)단위로 정렬하고 압축함
+ 블록은 여러 대의 머신이나 외부 메모리에 분산될 수 있음
+ 분할 탐색 알고리즘은 블록의 장점을 사용해 분위수 탐색을 빠르게 수행함
+ 해당 부분에 병렬 컴퓨팅을 사용하여 모델 구축 과정의 속도를 높일 수 있음


#### 2-4) 캐시 고려 접근
컴퓨터의 데이터는 캐시와 메인 메모리에 나뉘어 있습니다. 가장 빈번하게 사용되는 캐시는 고속 메모리를 사용합니다. 자주 사용하지 않는 데이터는 저속 메모리에 저장됩니다.

XGBoost는 캐시를 고려한 프리페칭(prefetching)을 사용하여, 많은 샘플을 가진 데이터셋의 실행 부하를 50% 절감

#### 3) 정확도 향상
XGBoost는 자체적으로 **규제**를 추가하여 Gradient Boosting 이상으로 정확도를 높임. 즉 **XGBoost는 Gradient Boosting의 규제 버전**
+ XGBoost는 Gradient Boosting과 RandomForest와 달리, 학습하려는 목적함수의 일부로 규제를 포함함
+ **규제**는 분산을 줄이고 과대적합을 방지함